# Interoperability between NumPy and external libraries

## Converting ndarray to PIL Image

*The following is based on an example by Pauli Virtanen in [Scipy lectures](http://www.scipy-lectures.org/advanced/advanced_numpy/index.html#interoperability-features).*

In [ ]:
from PIL import Image
import numpy as np
# In PIL, RGB images consist of 4-byte integers whose bytes are [RR,GG,BB, AA]
data = np.zeros((200, 200), dtype=[('r', np.uint8),
                                   ('g', np.uint8),
                                   ('b', np.uint8),
                                   ('a', np.uint8)])
img = Image.fromarray(data, mode='RGBA')
data['b'] =  255 # Red
data['a'] = 255
img

NumPy and PIL share the same block of memory!

In [ ]:
data['r']=255
img

## Converting PIL Image to ndarray

In [ ]:
from PIL import Image
img = Image.open('lena.jpg')
img

In [ ]:
arr = np.asarray(img)
arr.shape

Unfortunately, PIL gives only read-only acces to the memory block.

In [ ]:
# arr /= 2

## The \_\_array\_interface\_\_

Any object that exposes a suitable dictionary named
``__array_interface__`` may be converted to a NumPy array. This is
handy for exchanging data with external libraries. The array interface
has the following important keys (see
http://docs.scipy.org/doc/numpy/reference/arrays.interface.html):

 - **shape**: Tuple whose elements are the array size in each dimension.
 - **typestr**: A string providing the basic type of the homogenous array. It consists of 3 characters - endiannes, type and number of bytes)
 - **data**: (20495857, True); 2-tuple—pointer to data and boolean to
indicate whether memory is read-only
 - **strides**
 - **version**: 3

In [ ]:
a = np.arange(5)
a.__array_interface__

## Copy or view

Slicing returns view on the same array

In [ ]:
a[::2].__array_interface__['data']

Fancy indexing returns a copy

In [ ]:
a[[0, 2, 4]].__array_interface__['data']

## Exercise

*Original exercise by Stefan van der Walt and Juan Nunez-Iglesias. Modified by Bartosz Telenczuk.*
 
An author of a foreign package (included with the exercises as
``mutable_str.py``) provides a string class that
allocates its own memory:

```ipython
In [1]: from mutable_str import MutableString
In [2]: s = MutableString('abcde')
In [3]: print s
abcde
```

You'd like to view these mutable (*mutable* means the ability to modify in place)
strings as ndarrays, in order to manipulate the underlying memory.

Add an `__array_interface__` dictionary attribute to s, then convert s to an
ndarray. Numerically add "2" to the array (use the in-place operator ``+=``).

Then print the original string to ensure that its value was modified.

> **Hint:** Documentation for NumPy's ``__array_interface__``
  may be found [in the online docs](http://docs.scipy.org/doc/numpy/reference/arrays.interface.html).

Here's a skeleton outline:

In [ ]:
import numpy as np
from mutable_str import MutableString

s = MutableString('abcde')

# --- EDIT THIS SECTION ---

# Create an array interface to this foreign object
s.__array_interface__ = {'data' : 'FIXME', # tuple (ptr, is read_only?)
                         'shape' : 'FIXME',
                         'typestr' : 'FIXME', # typecode unsigned character
                         }

# --- EDIT THIS SECTION ---

print('String before converting to array:', s)
sa = np.asarray(s)

print('String after converting to array:', sa)

sa += 2
print('String after adding "2" to array:', s)


## Extra reading

* SciPy lectures: http://www.scipy-lectures.org/advanced/advanced_numpy/index.html#interoperability-features
* PEP 3118 -- Revising the buffer protocol: https://www.python.org/dev/peps/pep-3118/
* Introduction to buffer protocol: https://jakevdp.github.io/blog/2014/05/05/introduction-to-the-python-buffer-protocol/